# Computer Vision Project: Phase 2

### Imports

In [3]:
from PIL import Image
from PIL.ExifTags import TAGS
import cv2
import numpy as np
import glob
from scipy.optimize import least_squares

### Calculating the Intrinsic Matrix Manually

In [ ]:
def get_exif(image_path):
    img = Image.open(image_path)
    exif_data = img._getexif()
    if exif_data is None:
        raise ValueError("No EXIF data found in image.")

    exif = {}
    for tag_id, value in exif_data.items():
        tag = TAGS.get(tag_id, tag_id)
        exif[tag] = value

    return exif, img.size  # (width, height)


# -------------------------------------------------------------
# 2. Sensor size database (add your phone here)
#    Units: millimeters (mm)
# -------------------------------------------------------------
PHONE_SENSORS = {
    "iPhone 12": (5.6, 4.2),
    "iPhone 13": (5.6, 4.2),
    "iPhone 14": (5.6, 4.2),
    "Samsung S21": (5.64, 4.23),
    "Samsung S22": (5.64, 4.23),
    # Add your model here if needed
}


# -------------------------------------------------------------
# 3. Extract focal length & compute K
# -------------------------------------------------------------
def compute_intrinsics_from_exif(image_path, phone_model):
    exif, (W, H) = get_exif(image_path)

    # --- Extract focal length (in mm) ---
    if "FocalLength" not in exif:
        raise ValueError("FocalLength EXIF tag missing.")

    num, den = exif["FocalLength"]
    focal_mm = num / den  # e.g., 4.2 mm

    # --- Get sensor size for this phone ---
    if phone_model not in PHONE_SENSORS:
        raise ValueError(f"Add your phone model '{phone_model}' to PHONE_SENSORS dictionary.")

    sensor_w_mm, sensor_h_mm = PHONE_SENSORS[phone_model]

    # --- Compute fx, fy ---
    fx = focal_mm * (W / sensor_w_mm)
    fy = focal_mm * (H / sensor_h_mm)

    # --- Principal point ---
    cx = W / 2
    cy = H / 2

    # --- Build intrinsic matrix K ---
    K = np.array([
        [fx, 0,  cx],
        [0,  fy, cy],
        [0,  0,  1 ]
    ])

    return K, fx, fy, cx, cy, focal_mm


# -------------------------------------------------------------
# 4. RUN IT
# -------------------------------------------------------------
IMAGE_PATH = "YOUR_IMAGE_PATH_HERE.jpg"
PHONE_MODEL = "iPhone 14"   # <<< change this

K, fx, fy, cx, cy, focal_mm = compute_intrinsics_from_exif(IMAGE_PATH, PHONE_MODEL)

print("\n--- CAMERA INTRINSICS ---")
print("Focal Length:", focal_mm, "mm")
print("fx:", fx)
print("fy:", fy)
print("cx:", cx)
print("cy:", cy)
print("\nK =\n", K)

### Feature Extraction & Matching

In [ ]:
# Feature Extraction:
def extract_features(img):
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(img, None)
    pts = np.array([kp.pt for kp in keypoints], dtype=np.float32)
    return pts, descriptors


# Feature Matching:
def match_features(desc1, desc2):
    bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)
    matches = bf.knnMatch(desc1, desc2, k=2)

    good = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good.append((m.queryIdx, m.trainIdx))

    return np.array(good)

# Normalization:
def normalize_points(pts):
    pts_h = np.hstack([pts, np.ones((pts.shape[0], 1))])
    return (K_inv @ pts_h.T).T[:, :2]

### Computing Essential Matrix, Recovering Poses, and Triangulation:

In [ ]:
# Computing E:
def compute_essential(pts1, pts2):
    E, mask = cv2.findEssentialMat(
        pts1, pts2, K, cv2.RANSAC, 0.999, 1.0
    )
    return E, mask.ravel().astype(bool)

# Recovering Pose
def recover_pose(E, pts1, pts2):
    _, R, t, _ = cv2.recoverPose(E, pts1, pts2, K)
    return R, t.reshape(3)

# Triangulation
def triangulate(R1, t1, R2, t2, pts1, pts2):
    P1 = K @ np.hstack([R1, t1.reshape(3,1)])
    P2 = K @ np.hstack([R2, t2.reshape(3,1)])

    pts1_h = cv2.convertPointsToHomogeneous(pts1).reshape(-1,3)
    pts2_h = cv2.convertPointsToHomogeneous(pts2).reshape(-1,3)

    X = cv2.triangulatePoints(P1, P2, pts1_h.T[:2], pts2_h.T[:2])
    X = (X / X[3])[:3].T   # convert from homogeneous

    return X

### Perspective-n-Point Algorithm:

In [ ]:
def solve_pnp(pts3d, pts2d):
    _, Rvec, tvec, inliers = cv2.solvePnPRansac(
        pts3d, pts2d, K, None
    )
    R, _ = cv2.Rodrigues(Rvec)
    t = tvec.reshape(3)
    return R, t, inliers

### Bundle Adjustment

In [ ]:
def reprojection_error(params, n_cams, n_points, cam_idx, pt_idx, pts2d):
    """ Bundle Adjustment cost function """
    camera_params = params[:n_cams * 6].reshape((n_cams, 6))
    points_3d = params[n_cams * 6:].reshape((n_points, 3))

    proj = []
    for i in range(len(cam_idx)):
        cam = cam_idx[i]
        pt = pt_idx[i]

        rvec = camera_params[cam, :3]
        tvec = camera_params[cam, 3:].reshape(3,1)
        R, _ = cv2.Rodrigues(rvec)

        X = points_3d[pt]
        x = K @ (R @ X + tvec)
        x = x[:2] / x[2]

        proj.append(x)

    proj = np.array(proj).reshape(-1,2)
    return (proj - pts2d).ravel()


def run_bundle_adjustment(camera_params, points_3d, cam_idx, pt_idx, pts2d):
    x0 = np.hstack((camera_params.ravel(), points_3d.ravel()))

    res = least_squares(
        reprojection_error,
        x0,
        verbose=1,
        x_scale='jac',
        ftol=1e-4,
        method='lm',
        args=(camera_params.shape[0], points_3d.shape[0], cam_idx, pt_idx, pts2d)
    )

    return res.x

## Entire SfM Pipeline

In [ ]:
def run_sfm(image_folder):
    # --- Load images ---
    paths = sorted(glob.glob(image_folder + "/*.jpg"))
    imgs = [cv2.imread(p) for p in paths]

    keypoints = []
    descriptors = []
    for img in imgs:
        pts, desc = extract_features(img)
        keypoints.append(pts)
        descriptors.append(desc)

    # Store all camera poses
    Rs = []
    ts = []

    # Store global 3D points
    global_points = []
    global_colors  = []

    point_map = {}

    matches = match_features(descriptors[0], descriptors[1])

    pts1 = keypoints[0][matches[:,0]]
    pts2 = keypoints[1][matches[:,1]]

    E, mask = compute_essential(pts1, pts2)
    pts1 = pts1[mask]
    pts2 = pts2[mask]

    R, t = recover_pose(E, pts1, pts2)

    Rs.append(np.eye(3))
    ts.append(np.zeros(3))

    Rs.append(R)
    ts.append(t)

    X = triangulate(Rs[0], ts[0], Rs[1], ts[1], pts1, pts2)

    for i, P in enumerate(X):
        point_map[(0, matches[mask][:,0][i])] = len(global_points)
        point_map[(1, matches[mask][:,1][i])] = len(global_points)
        global_points.append(P)

    for i in range(2, len(imgs)):
        print(f"\n=== PROCESSING IMAGE {i} ===")

        pts2d = []
        pts3d = []

        # find 2D–3D correspondences
        for prev in range(i):
            matches = match_features(descriptors[prev], descriptors[i])

            for (idx_prev, idx_curr) in matches:
                if (prev, idx_prev) in point_map:
                    pts2d.append(keypoints[i][idx_curr])
                    pts3d.append(global_points[point_map[(prev, idx_prev)]])

        pts2d = np.array(pts2d, dtype=np.float32)
        pts3d = np.array(pts3d, dtype=np.float32)

        # must have enough to solve PnP
        if len(pts3d) < 20:
            print("Not enough correspondences.")
            continue

        R, t, _ = solve_pnp(pts3d, pts2d)
        Rs.append(R)
        ts.append(t)

        # triangulate new points
        for prev in range(i):
            matches = match_features(descriptors[prev], descriptors[i])

            pts_prev = keypoints[prev]
            pts_curr = keypoints[i]

            ptsA = []
            ptsB = []
            idxA = []
            idxB = []

            for (idx_prev, idx_curr) in matches:
                if (prev, idx_prev) not in point_map:
                    ptsA.append(pts_prev[idx_prev])
                    ptsB.append(pts_curr[idx_curr])
                    idxA.append(idx_prev)
                    idxB.append(idx_curr)

            if len(ptsA) == 0:
                continue

            Xnew = triangulate(
                Rs[prev], ts[prev],
                R, t,
                np.array(ptsA), np.array(ptsB)
            )

            for j, P in enumerate(Xnew):
                global_points.append(P)
                pid = len(global_points)-1
                point_map[(prev, idxA[j])] = pid
                point_map[(i, idxB[j])] = pid

    with open("output_sfm.ply", "w") as f:
        f.write("ply\nformat ascii 1.0\nelement vertex {}\n".format(len(global_points)))
        f.write("property float x\nproperty float y\nproperty float z\nend_header\n")
        for p in global_points:
            f.write("{} {} {}\n".format(p[0], p[1], p[2]))

    print("\nSFM COMPLETE. Saved output_sfm.ply")

    return Rs, ts, global_points